In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import os
import torch

In [2]:
spx_vol_df = pd.read_csv('../data/finance_data/spx_implied_vol_raw.csv')
print('Number of Records:', len(spx_vol_df))
spx_vol_df.head()

Number of Records: 1086844


,secid,date,days,delta,impl_volatility,impl_strike,impl_premium,dispersion,cp_flag,ticker,sic,index_flag
0,108105,1999-12-27,10,10,NaN,0.0,0.0,NaN,C,SPX,9999,1
1,108105,1999-12-27,10,15,NaN,0.0,0.0,NaN,C,SPX,9999,1
2,108105,1999-12-27,10,20,NaN,0.0,0.0,NaN,C,SPX,9999,1
3,108105,1999-12-27,10,25,NaN,0.0,0.0,NaN,C,SPX,9999,1
4,108105,1999-12-27,10,30,NaN,0.0,0.0,NaN,C,SPX,9999,1


In [3]:
# Check for missing values
missing_val_df = spx_vol_df[spx_vol_df['impl_volatility'].isnull()]

# Get configurations of days to expiration and delta which has missing values
missing_val_configurations = missing_val_df[['days','delta']].drop_duplicates().values
missing_val_configurations

array([[10, 10],
       [10, 15],
       [10, 20],
       [10, 25],
       [10, 30],
       [10, 35],
       [10, 40],
       [10, 45],
       [10, 50],
       [10, 55],
       [10, 60],
       [10, 65],
       [10, 70],
       [10, 75],
       [10, 80],
       [10, 85],
       [10, 90]], dtype=int64)

In [4]:
spx_vol_df_final = spx_vol_df.copy()

In [5]:
# Do linear interpolation for null values
for d in missing_val_configurations:
    tmp = spx_vol_df_final[(spx_vol_df_final['days'] == d[0]) & (spx_vol_df_final['delta'] == d[1])]
    tmp['impl_volatility'] = tmp['impl_volatility'].interpolate()
    
    # index of these
    index_to_update = tmp.index
    
    # Update
    spx_vol_df_final.loc[index_to_update, 'impl_volatility'] = tmp['impl_volatility']

C:\Users\angsi\AppData\Local\Temp\ipykernel_9088\3540626349.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['impl_volatility'] = tmp['impl_volatility'].interpolate()


In [6]:
spx_vol_df_final[spx_vol_df_final['impl_volatility'].isnull()]['date'].max()

'2005-11-03'

In [7]:
# In view of the above, we will use data from 2006 to 2022
spx_vol_df_final = spx_vol_df_final[spx_vol_df_final['date'].apply(lambda x: (int(x[:4]) >= 2006) and (int(x[:4]) <= 2022))]

# I will also pick the same set of days to expiry and delta as the paper
# Delta = (0.1; 0.25; 0.5; 0.75; and 0.9), Days to Expiry = (ten days, one month, three and six months, and one year)
# Total 25 points per day
spx_vol_df_final = spx_vol_df_final[(spx_vol_df_final['days'].isin([10,30,91,182,365])) & (spx_vol_df_final['delta'].isin([10,25,50,75,90]))]

spx_vol_df_final = spx_vol_df_final.reset_index(drop = True)
spx_vol_df_final

,secid,date,days,delta,impl_volatility,impl_strike,impl_premium,dispersion,cp_flag,ticker,sic,index_flag
0,108105,2006-01-03,10,10,0.099910,0.000,0.00000,NaN,C,SPX,9999,1
1,108105,2006-01-03,10,25,0.094388,0.000,0.00000,NaN,C,SPX,9999,1
2,108105,2006-01-03,10,50,0.101147,0.000,0.00000,NaN,C,SPX,9999,1
3,108105,2006-01-03,10,75,0.118595,0.000,0.00000,NaN,C,SPX,9999,1
4,108105,2006-01-03,10,90,0.211980,0.000,0.00000,NaN,C,SPX,9999,1
...,...,...,...,...,...,...,...,...,...,...,...,...
106970,108105,2022-12-30,365,10,0.169997,5027.882,28.91882,0.005779,C,SPX,9999,1
106971,108105,2022-12-30,365,25,0.184180,4587.279,96.83470,0.008266,C,SPX,9999,1
106972,108105,2022-12-30,365,50,0.215120,4070.207,291.55340,0.009880,C,SPX,9999,1
106973,108105,2022-12-30,365,75,0.250455,3452.004,665.31250,0.011261,C,SPX,9999,1


In [8]:
spx_vol_df_final.to_csv('../data/finance_data/spx_implied_vol_interpolated.csv', index = False)